In [1]:
# Add a delay before registering an object (Must pass below a treshold)
# Deregister if below a certain treshold for N number of frames 
# Do not let the objects to match if the distance is above sertain treshold 

In [2]:
from scipy.spatial import distance as dist
from collections import OrderedDict
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt

class CentroidTracker(): # ok?
    def __init__(self, maxDisappeared=10, upperTreshold = 50, lowerTreshold = 50, minCandiateFrames = 10, maxDist = 40):
        # initialize the next unique object ID along with two ordered
        # dictionaries used to keep track of mapping a given object
        # ID to its centroid and number of consecutive frames it has
        # been marked as "disappeared", respectively
        self.nextObjectID = 1
        self.objects = OrderedDict()
        self.disappeared = OrderedDict()
        
        # @aydamir
        self.maxDist = maxDist
        self.candidateObjectID = -1
        self.minCandidateFrames = 10
        self.candidateObjects = OrderedDict()
        self.candidateObjectScores = OrderedDict()

        # store the number of maximum consecutive frames a given
        # object is allowed to be marked as "disappeared" until we
        # need to deregister the object from tracking
        self.maxDisappeared = maxDisappeared
        
        # @aydamir
        self.upperTreshold = upperTreshold
        self.lowerTreshold = lowerTreshold

    def register(self, centroid): # ok?
        # when registering an object we use the next available object
        # ID to store the centroid
        
        self.objects[self.nextObjectID] = centroid  
        self.disappeared[self.nextObjectID] = 0  # This keeps count of object dissapearance.
        self.nextObjectID += 1
    
    # @aydamir
    def add_candidate(self, centroid):
        # when new object enters the scene, instead of adding it as a new object add it as
        # an object candidate and if it stays in for more than N number of frames 
        # register it as a new permament object
        
        self.candidateObjects[self.candidateObjectID] = centroid
        self.candidateObjectScores[self.candidateObjectID] = 1
        self.candidateObjectID -= 1

    def remove_candidate(self, objectID):
        # Deregister a candidate object
        
        del self.candidateObjects[objectID]
        del self.candidateObjectScores[objectID]
    
    def deregister(self, objectID): # ok
        # to deregister an object ID we delete the object ID from
        # both of our respective dictionaries
        del self.objects[objectID]
        del self.disappeared[objectID]
    
    def update(self, rects): 
        # check to see if the list of input bounding box rectangles
        # is empty
        if len(rects) == 0:
            # loop over any existing tracked objects and mark them
            # as disappeared
            for objectID in list(self.disappeared.keys()):
                self.disappeared[objectID] += 1

                # if we have reached a maximum number of consecutive
                # frames where a given object has been marked as
                # missing, deregister it
                if self.disappeared[objectID] > self.maxDisappeared:
                    self.deregister(objectID)
                    
            # @aydamir
            # Also clear the list of candidate objects 
            for objectID in list(self.candidateObjects.keys()):
                self.remove_candidate(objectID)

            # return early as there are no centroids or tracking info
            # to update
            return self.objects

        # initialize an array of input centroids for the current frame
        inputCentroids = np.zeros((len(rects), 2), dtype="int")

        # loop over the bounding box rectangles
        for (i, (startX, startY, endX, endY)) in enumerate(rects):
            # use the bounding box coordinates to derive the centroid
            cX = int((startX + endX) / 2.0)
            cY = int((startY + endY) / 2.0)
            inputCentroids[i] = (cX, cY)

        # if there are currently not any candidate objects 
        # add inputs to the list of candidate objects
        #print(len(self.candidateObjects), len(self.objects))
        if len(self.candidateObjects) == 0 and len(self.objects) == 0:
            for i in range(0, len(inputCentroids)):
                self.add_candidate(inputCentroids[i])
                
        
#         # if we are currently not tracking any objects take the input
#         # centroids and register each of them
#         if len(self.objects) == 0:
#             for i in range(0, len(inputCentroids)):
#                 self.register(inputCentroids[i])

        # otherwise, if we are currently tracking objects we need to
        # try to match the input centroids to existing object
        # centroids
        else:
            # grab the set of object IDs and candidate IDs corresponding centroids
            objectIDs = list(self.objects.keys()) + list(self.candidateObjects.keys())
            objectCentroids = list(self.objects.values()) + list(self.candidateObjects.values())

            # compute the distance between each pair of object
            # centroids and input centroids, respectively -- our
            # goal will be to match an input centroid to an existing
            # object centroid
            D = dist.cdist(np.array(objectCentroids), inputCentroids)

            # in order to perform this matching we must (1) find the
            # smallest value in each row and then (2) sort the row
            # indexes based on their minimum values so that the row
            # with the smallest value as at the *front* of the index
            # list
            rows = D.min(axis=1).argsort()

            # next, we perform a similar process on the columns by
            # finding the smallest value in each column and then
            # sorting using the previously computed row index list
            cols = D.argmin(axis=1)[rows]
    
            # in order to determine if we need to update, register,
            # or deregister an object we need to keep track of which
            # of the rows and column indexes we have already examined
            usedRows = set()
            usedCols = set()

            # loop over the combination of the (row, column) index
            # tuples
            for (row, col) in zip(rows, cols):
                # if we have already examined either the row or
                # column value before, ignore it
                # val
                if row in usedRows or col in usedCols:
                    continue
                    
                if D[row, col] < self.maxDist:
                    usedRows.add(row)
                    usedCols.add(col)
                    continue

                # otherwise, grab the object ID for the current row,
                # set its new centroid, and reset the disappeared
                # counter
                objectID = objectIDs[row]
                
                if objectID > 0: # If the object is candidate   
                    #print('get hehehehe')
                    self.objects[objectID] = inputCentroids[col]
                    self.disappeared[objectID] = 0
                else:
                    self.candidateObjects[objectID] = inputCentroids[col]
                    self.candidateObjectScores[objectID] += 1
                    
                    # If object has been present for more than N number of frames 
                    # Add it to the list of objects
                    if self.candidateObjectScores[objectID] >= self.minCandidateFrames  and inputCentroids[col][1] > self.upperTreshold:
                        #print('Get here')
                        self.register(inputCentroids[col])
                        self.remove_candidate(objectID)

                # indicate that we have examined each of the row and
                # column indexes, respectively
                usedRows.add(row)
                usedCols.add(col)

            # compute both the row and column index we have NOT yet
            # examined
            unusedRows = set(range(0, D.shape[0])).difference(usedRows)
            unusedCols = set(range(0, D.shape[1])).difference(usedCols)

            # in the event that the number of object centroids is
            # equal or greater than the number of input centroids
            # we need to check and see if some of these objects have
            # potentially disappeared
            if D.shape[0] >= D.shape[1]:
                # loop over the unused row indexes
                for row in unusedRows:
                    # grab the object ID for the corresponding row
                    # index and increment the disappeared counter
                    objectID = objectIDs[row]
                    
                    if objectID > 0:
                        self.disappeared[objectID] += 1
                        # check to see if the number of consecutive
                        # frames the object has been marked "disappeared"
                        # for warrants deregistering the object
                        if self.disappeared[objectID] > self.maxDisappeared:
                            self.deregister(objectID)
                    else:
                        self.remove_candidate(objectID)

            # otherwise, if the number of input centroids is greater
            # than the number of existing object centroids we need to
            # register each new input centroid as a trackable object
            else:
                for col in unusedCols:
                    self.add_candidate(inputCentroids[col])
                    #self.register(inputCentroids[col])

        # return the set of trackable objects
        return self.objects

In [3]:
# @aydamir
def convert_yolo_to_COCO(bbox, im_height, im_width):
    COCO_bbox = []
    
    x_center = bbox[0]
    y_center = bbox[1]
    width = int(round(bbox[2]*im_width))
    height = int(round(bbox[3]*im_height))
    
    xmin = (x_center*2*im_width - width)/2
    xmin = int(round(xmin))
    ymin = (y_center*2*im_height - height)/2
    ymin = int(round(ymin))

    COCO_bbox.append(xmin)
    COCO_bbox.append(ymin)
    COCO_bbox.append(xmin + width)
    COCO_bbox.append(ymin + height)

    return COCO_bbox

#@aydamir
def get_labels_coordinates(lines):
    nut_labels = []
    nut_coordinates = []
    
    bolt_labels = []
    bolt_coordinates = []

    for line in lines:
        
        label = int(line[0])
        bbox = []
        for cor in line[2:].split(' '):
            bbox.append(float(cor))
        bbox = convert_yolo_to_COCO(bbox, 640, 640)
            
        if label == 0:
            bolt_coordinates.append(bbox)
            bolt_labels.append(label)
        else:
            nut_coordinates.append(bbox)
            nut_labels.append(label)
        
    data = {}
    
    data['bolt_labels'] = bolt_labels
    data['bolt_bboxes'] = bolt_coordinates
    data['nut_labels'] = nut_labels
    data['nut_bboxes'] = nut_coordinates
         
    return data

In [47]:
point1 = 'C:/Users/mirza/Desktop/stroma-tracking/experiments/exp3/labels/0700.txt'
point2 = 'C:/Users/mirza/Desktop/stroma-tracking/experiments/exp3/labels/0701.txt'


with open( point1) as f:
    lines = f.readlines()
f.close()
point1 = get_labels_coordinates(lines)


with open( point2) as f:
    lines = f.readlines()
f.close()
point2 = get_labels_coordinates(lines)

In [5]:
point1 = [[20,21], [45,48], [90,78], [200,500]]
point2 = [[45,50], [90,82], [20,24]]

In [6]:
D = dist.cdist(np.array(point1), np.array(point2))

In [9]:
D[0,0]

38.28837943815329

In [74]:
rows = D.min(axis=1).argsort()

In [75]:
rows

array([1, 0, 2, 3], dtype=int64)

In [78]:
cols = D.argmin(axis=1)[rows]

In [79]:
cols

array([0, 2, 1, 1], dtype=int64)

In [4]:
nut_centroids = CentroidTracker(maxDisappeared=5, upperTreshold = 100, minCandiateFrames = 15)
bolt_centroids = CentroidTracker(maxDisappeared=5, upperTreshold = 100, minCandiateFrames = 15)

source_folder = 'C:/Users/mirza/Desktop/stroma-tracking/experiments/exp3/labels'
frame_source = 'C:/Users/mirza/Desktop/stroma-tracking/experiments/exp3'
target_frame_source = 'C:/Users/mirza/Desktop/stroma-tracking/experiments/exp3/counted_objects'

frame_boxes = []
for file in os.listdir(source_folder):
    
    with open( source_folder + '/' + file) as f:
        lines = f.readlines()
    f.close()
    
    out = get_labels_coordinates(lines)
    frame_boxes.append(out)
    
    
    bolts = bolt_centroids.update(out['bolt_bboxes'])
    nuts = nut_centroids.update(out['nut_bboxes'])
    
    name = file.split('.')[0]
    
    frame = cv2.imread(f'{frame_source}/{name}.jpg')
    
    #print(len(objects.items()), len(out['bboxes']))
    
    for i, (objectID, centroid) in enumerate(bolts.items()):
        text = "Bolt: {}".format(objectID)
        cv2.putText(frame, text, (centroid[0] - 40, centroid[1]-40), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        
    for i, (objectID, centroid) in enumerate(nuts.items()):
        text = "Nut:  {}".format(objectID)
        cv2.putText(frame, text, (centroid[0] - 40, centroid[1]-40), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    
    cv2.imwrite(f'{target_frame_source}/{name}.jpg', frame)
    

In [20]:
out = {'labels': [0, 0, 1], 'bboxes': [[159, 0, 230, 60],[120, 20, 50, 60],[190, 60, 30, 120]]}

In [21]:
out['labels'][out['labels']==]

0